In [153]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [154]:

# URL of the Basketball Reference contracts page
url = "https://www.basketball-reference.com/contracts/players.html"
# Step 1: Get the page HTML
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
# Step 2: Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Find the contracts table
table = soup.find("table", {"id": "player-contracts"})

# Step 4: Read table into pandas
contract25 = pd.read_html(str(table))[0]

from IPython.display import display
contract25.columns = [
    "Rk", "Player", "Tm", "2024-25", "2025-26", "2026-27", "2027-28", "2028-29", "2029-30", "Guaranteed"
]
contract25 = contract25[contract25["Player"].notna() & (contract25["Player"] != "Player") & (contract25["Player"] != "Missing value")]
contract25['Salary Count'] = contract25.iloc[:, 3:9].notnull().sum(axis=1)
# 1. Make sure it's a string, strip whitespace
contract25['Guaranteed'] = contract25['Guaranteed'].astype(str).str.strip()

# 2. Remove dollar signs and commas (or any non‑digit/dot)
contract25['Guaranteed'] = contract25['Guaranteed']\
    .str.replace(r'[^0-9\.]', '', regex=True)

# 3. Now convert to float
contract25['Guaranteed'] = pd.to_numeric(contract25['Guaranteed'], errors='coerce')

# 4. Check for anything that still failed to convert
bad = contract25['Guaranteed'].isna()
if bad.any():
    print("These rows are still non‑numeric:", df.loc[bad, 'Guaranteed'])
# 5. Finally compute your float average
contract25['Average Salary'] = contract25['Guaranteed'] / contract25['Salary Count']

C:\Users\girth\AppData\Local\Temp\ipykernel_22664\302353211.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  contract25 = pd.read_html(str(table))[0]


In [ ]:
'''
# URL of the archived Basketball‑Reference contracts page
url = "https://web.archive.org/web/20240603223914/https://www.basketball-reference.com/contracts/players.html"

# Step 1: Get the page HTML
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

# Step 2: Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Find the contracts table
table = soup.find("table", {"id": "player-contracts"})

# Step 4: Read table into pandas
contract24 = pd.read_html(str(table))[0]

# Step 5: Clean up column names
contract24.columns = [
    "Rk", "Player", "Tm",
    "2024-25", "2025-26", "2026-27", "2027-28", "2028-29", "2029-30",
    "Guaranteed"
]

# Step 6: Drop header‑repeats and any blanks
contract24 = contract24[
    contract24["Player"].notna()
    & (contract24["Player"] != "Player")
    & (contract24["Player"] != "Missing value")
].copy()

# Step 7: Count how many of the year columns are non‑null
contract24['Salary Count'] = contract24.iloc[:, 3:9].notnull().sum(axis=1).astype(int)

# Step 8: Clean and convert the "Guaranteed" column to float
contract24['Guaranteed'] = (
    contract24['Guaranteed']
      .astype(str)
      .str.replace(r'[^0-9\.]', '', regex=True)
)
contract24['Guaranteed'] = pd.to_numeric(contract24['Guaranteed'], errors='coerce')

# Optional sanity check: see if any rows still failed
bad = contract24['Guaranteed'].isna()
if bad.any():
    print("Rows with non‑numeric Guaranteed:", contract24.loc[bad, ['Player','Guaranteed']])

# Step 9: Compute average salary as a float
contract24['Average Salary'] = contract24['Guaranteed'] / contract24['Salary Count']
'''

Rows with non‑numeric Guaranteed:                 Player  Guaranteed
320  Derrick Jones Jr.         NaN
448        Moses Brown         NaN
449   Boban Marjanović         NaN
450      Aaron Holiday         NaN
451   Tristan Thompson         NaN
452    Trendon Watford         NaN
455   Garrison Mathews         NaN
458         Isaiah Joe         NaN
480      Aaron Wiggins         NaN
481      Jose Alvarado         NaN
488        Skylar Mays         NaN


C:\Users\girth\AppData\Local\Temp\ipykernel_22664\82644973.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  contract24 = pd.read_html(str(table))[0]


In [ ]:
'''
# URL of the archived Basketball‑Reference contracts page
url = "https://web.archive.org/web/20240603223914/https://www.basketball-reference.com/contracts/players.html"

# Step 1: Get the page HTML
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

# Step 2: Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Find the contracts table
table = soup.find("table", {"id": "player-contracts"})

# Step 4: Read table into pandas
contract24 = pd.read_html(str(table))[0]

# Step 5: Clean up column names
contract24.columns = [
    "Rk", "Player", "Tm",
    "2023-24", "2024-25", "2025-26", "2026-27", "2027-28", "2028-29",
    "Guaranteed"
]

# Step 6: Drop header‑repeats and any blanks
contract24 = contract24[
    contract24["Player"].notna()
    & (contract24["Player"] != "Player")
    & (contract24["Player"] != "Missing value")
].copy()

# Step 7: Count how many of the year columns are non‑null
contract24['Salary Count'] = contract24.iloc[:, 3:9].notnull().sum(axis=1).astype(int)

# Step 8: Clean and convert the "Guaranteed" column to float
contract24['Guaranteed'] = (
    contract24['Guaranteed']
      .astype(str)
      .str.replace(r'[^0-9\.]', '', regex=True)
)
contract24['Guaranteed'] = pd.to_numeric(contract24['Guaranteed'], errors='coerce')

# Optional sanity check: see if any rows still failed
bad = contract24['Guaranteed'].isna()
if bad.any():
    print("Rows with non‑numeric Guaranteed:", contract24.loc[bad, ['Player','Guaranteed']])

# Step 9: Compute average salary as a float
contract24['Average Salary'] = contract24['Guaranteed'] / contract24['Salary Count']
'''

Rows with non‑numeric Guaranteed:                 Player  Guaranteed
320  Derrick Jones Jr.         NaN
448        Moses Brown         NaN
449   Boban Marjanović         NaN
450      Aaron Holiday         NaN
451   Tristan Thompson         NaN
452    Trendon Watford         NaN
455   Garrison Mathews         NaN
458         Isaiah Joe         NaN
480      Aaron Wiggins         NaN
481      Jose Alvarado         NaN
488        Skylar Mays         NaN


C:\Users\girth\AppData\Local\Temp\ipykernel_22664\2929837399.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  contract24 = pd.read_html(str(table))[0]


In [ ]:
'''
# URL of the archived Basketball‑Reference contracts page
url = "https://web.archive.org/web/20230516051257/https://www.basketball-reference.com/contracts/players.html"

# Step 1: Get the page HTML
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

# Step 2: Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Find the contracts table
table = soup.find("table", {"id": "player-contracts"})

# Step 4: Read table into pandas
contract23 = pd.read_html(str(table))[0]

# Step 5: Clean up column names
contract23.columns = [
    "Rk", "Player", "Tm",
    "2022-23", "2023-24", "2024-25", "2025-26", "2026-27", "2027-28",
    "Guaranteed"
]

# Step 6: Drop header‑repeats and any blanks
contract23 = contract23[
    contract23["Player"].notna()
    & (contract23["Player"] != "Player")
    & (contract23["Player"] != "Missing value")
].copy()

# Step 7: Count how many of the year columns are non‑null
contract23['Salary Count'] = contract23.iloc[:, 3:9].notnull().sum(axis=1).astype(int)

# Step 8: Clean and convert the "Guaranteed" column to float
contract23['Guaranteed'] = (
    contract23['Guaranteed']
      .astype(str)
      .str.replace(r'[^0-9\.]', '', regex=True)
)
contract23['Guaranteed'] = pd.to_numeric(contract23['Guaranteed'], errors='coerce')

# Optional sanity check: see if any rows still failed
bad = contract23['Guaranteed'].isna()
if bad.any():
    print("Rows with non‑numeric Guaranteed:", contract23.loc[bad, ['Player','Guaranteed']])

# Step 9: Compute average salary as a float
contract23['Average Salary'] = contract23['Guaranteed'] / contract23['Salary Count']

return contract
'''

C:\Users\girth\AppData\Local\Temp\ipykernel_22664\2020503456.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  contract23 = pd.read_html(str(table))[0]


In [188]:
def load_contracts(url, years):
    """
    Fetches the Basketball‑Reference contracts table at `url`,
    cleans it, and computes Salary Count & Average Salary.
    
    years: list of the six season‑column names, e.g.
           ["2022-23","2023-24","2024-25","2025-26","2026-27","2027-28"]
    """
    # Step 1–3: get & parse
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table", {"id": "player-contracts"})
    
    # Step 4: read into pandas
    df = pd.read_html(str(table))[0]
    
    # Step 5: set cols
    df.columns = ["Rk", "Player", "Tm"] + years + ["Guaranteed"]
    
    # Step 6: drop repeats/blanks
    df = df[
        df["Player"].notna()
        & (df["Player"] != "Player")
        & (df["Player"] != "Missing value")
    ].copy()
    
    # Step 7: count non-null salaries
    df['Salary Count'] = (
        df.iloc[:, 3:3+len(years)]
          .notnull()
          .sum(axis=1)
          .astype(int)
    )
    
    # Step 8: clean & convert Guaranteed to float
    df['Guaranteed'] = (
        df['Guaranteed']
          .astype(str)
          .str.replace(r'[^0-9\.]', '', regex=True)
    )
    df['Guaranteed'] = pd.to_numeric(df['Guaranteed'], errors='coerce')
    
    # report any failures
    bad = df['Guaranteed'].isna()
    if bad.any():
        print("Rows with non‑numeric Guaranteed:",
              df.loc[bad, ['Player','Guaranteed']])
    
    # Step 9: compute average
    df['Average Salary'] = df['Guaranteed'] / df['Salary Count']
    
    return df

# --- now just call it for each snapshot:

years23 = ["2022-23","2023-24","2024-25","2025-26","2026-27","2027-28"]
url23   = "https://web.archive.org/web/20230516051257/https://www.basketball-reference.com/contracts/players.html"
contract23 = load_contracts(url23, years23)

years24 = ["2023-24","2024-25","2025-26","2026-27","2027-28","2028-29"]
url24   = "https://web.archive.org/web/20240603223914/https://www.basketball-reference.com/contracts/players.html"
contract24 = load_contracts(url24, years24)
display(contract23)
display(contract24)

C:\Users\girth\AppData\Local\Temp\ipykernel_22664\3197281329.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Rows with non‑numeric Guaranteed:                 Player  Guaranteed
320  Derrick Jones Jr.         NaN
448        Moses Brown         NaN
449   Boban Marjanović         NaN
450      Aaron Holiday         NaN
451   Tristan Thompson         NaN
452    Trendon Watford         NaN
455   Garrison Mathews         NaN
458         Isaiah Joe         NaN
480      Aaron Wiggins         NaN
481      Jose Alvarado         NaN
488        Skylar Mays         NaN


C:\Users\girth\AppData\Local\Temp\ipykernel_22664\3197281329.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Rk,Player,Tm,2022-23,2023-24,2024-25,2025-26,2026-27,2027-28,Guaranteed,Salary Count,Average Salary
0,1,Stephen Curry,GSW,"$48,070,014","$51,915,615","$55,761,216","$59,606,817",NaN,NaN,215353662,4,5.383842e+07
1,2,Russell Westbrook,UTA,"$47,559,433",NaN,NaN,NaN,NaN,NaN,47063478,1,4.706348e+07
2,3,LeBron James,LAL,"$44,474,988","$53,369,986","$57,639,585",NaN,NaN,NaN,97844974,3,3.261499e+07
3,4,Kevin Durant,PHO,"$44,119,845","$47,649,433","$51,179,021","$54,708,609",NaN,NaN,197656908,4,4.941423e+07
4,5,Bradley Beal,WAS,"$43,279,250","$46,741,590","$50,203,930","$53,666,270","$57,128,610",NaN,193891040,5,3.877821e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
617,562,Charles Bassey,PHI,"$2,674,742","$2,600,000","$2,500,000","$2,500,000",NaN,NaN,74742,4,1.868550e+04
618,563,Trevor Keels,NYK,NaN,NaN,NaN,NaN,NaN,NaN,58493,0,inf
619,564,Lester Quinones,GSW,NaN,NaN,NaN,NaN,NaN,NaN,58493,0,inf
620,565,Stanley Umude,DET,"$58,493",NaN,NaN,NaN,NaN,NaN,58493,1,5.849300e+04


,Rk,Player,Tm,2023-24,2024-25,2025-26,2026-27,2027-28,2028-29,Guaranteed,Salary Count,Average Salary
0,1,Stephen Curry,GSW,"$51,915,615","$55,761,216","$59,606,817",NaN,NaN,NaN,167283648.0,3,55761216.0
1,2,Kevin Durant,PHO,"$47,649,433","$51,179,021","$54,708,609",NaN,NaN,NaN,153537063.0,3,51179021.0
2,3,Nikola Jokić,DEN,"$47,607,350","$51,415,938","$55,224,526","$59,033,114","$62,841,702",NaN,213280928.0,5,42656185.6
3,4,Joel Embiid,PHI,"$47,607,350","$51,415,938","$55,224,526","$59,033,114",NaN,NaN,154247814.0,4,38561953.5
4,5,LeBron James,LAL,"$47,607,350","$51,415,938",NaN,NaN,NaN,NaN,47607350.0,2,23803675.0
...,...,...,...,...,...,...,...,...,...,...,...,...
611,558,Matthew Hurt,MEM,NaN,NaN,NaN,NaN,NaN,NaN,64343.0,0,inf
612,559,Pete Nance,CLE,"$64,343",NaN,NaN,NaN,NaN,NaN,64343.0,1,64343.0
613,560,Malik Williams,TOR,"$77,212",NaN,NaN,NaN,NaN,NaN,77212.0,1,77212.0
616,561,Kai Jones,LAC,"$3,175,563","$2,196,970",NaN,NaN,NaN,NaN,11608.0,2,5804.0


In [187]:
# URL of the archived Basketball‑Reference contracts page
url = "https://web.archive.org/web/20220127184320/https://www.basketball-reference.com//contracts/players.html"

# Step 1: Get the page HTML
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

# Step 2: Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Find the contracts table
table = soup.find("table", {"id": "player-contracts"})

# Step 4: Read table into pandas
contract22 = pd.read_html(str(table))[0]

# Step 5: Clean up column names
contract22.columns = [
    "Rk", "Player", "Tm",
    "2021-22", "2022-23", "2023-24", "2024-25", "2025-26", "2026-27", "Bird Rights",
    "Guaranteed"
]

# Step 6: Drop header‑repeats and any blanks
contract22 = contract22[
    contract22["Player"].notna()
    & (contract22["Player"] != "Player")
    & (contract22["Player"] != "Missing value")
].copy()

# Step 7: Count how many of the year columns are non‑null
contract22['Salary Count'] = contract22.iloc[:, 3:9].notnull().sum(axis=1).astype(int)

# Step 8: Clean and convert the "Guaranteed" column to float
contract22['Guaranteed'] = (
    contract22['Guaranteed']
      .astype(str)
      .str.replace(r'[^0-9\.]', '', regex=True)
)
contract22['Guaranteed'] = pd.to_numeric(contract22['Guaranteed'], errors='coerce')

# Optional sanity check: see if any rows still failed
bad = contract22['Guaranteed'].isna()
if bad.any():
    print("Rows with non‑numeric Guaranteed:", contract22.loc[bad, ['Player','Guaranteed']])

# Step 9: Compute average salary as a float
contract22['Average Salary'] = contract22['Guaranteed'] / contract22['Salary Count']
display(contract22)

Rows with non‑numeric Guaranteed:                       Player  Guaranteed
336            Avery Bradley         NaN
337                 Ed Davis         NaN
397         Kenrich Williams         NaN
399             Damian Jones         NaN
400          Wesley Matthews         NaN
404  Timothé Luwawu-Cabarrot         NaN
406               Damion Lee         NaN
410            Jalen Brunson         NaN
415         Dennis Smith Jr.         NaN
427              Isaiah Roby         NaN
428                 Naz Reid         NaN
429            Jaylen Nowell         NaN
430            Vlatko Čančar         NaN
431                Dean Wade         NaN
432          Jalen McDaniels         NaN
434            Chimezie Metu         NaN
435            Yuta Watanabe         NaN
442              Isaac Bonga         NaN
443       Isaiah Hartenstein         NaN
445           Oshae Brissett         NaN
446              Moses Brown         NaN
447       Isaiah Hartenstein         NaN
470             Anthony

C:\Users\girth\AppData\Local\Temp\ipykernel_22664\3228506678.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  contract22 = pd.read_html(str(table))[0]


,Rk,Player,Tm,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27,Bird Rights,Guaranteed,Salary Count,Average Salary
0,1,Stephen Curry,GSW,"$45,780,966","$48,070,014","$51,915,615","$55,761,216","$59,606,817",NaN,Bird Rights,261134628.0,5,52226925.6
1,2,John Wall,HOU,"$44,310,840","$47,366,760",NaN,NaN,NaN,NaN,Bird Rights,44310840.0,2,22155420.0
2,3,Russell Westbrook,LAL,"$44,211,146","$47,063,478",NaN,NaN,NaN,NaN,Bird Rights,44211146.0,2,22105573.0
3,4,James Harden,BRK,"$43,848,000","$46,872,000",NaN,NaN,NaN,NaN,Bird Rights,43848000.0,2,21924000.0
4,5,LeBron James,LAL,"$41,180,544","$44,474,988",NaN,NaN,NaN,NaN,Bird,85655532.0,2,42827766.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,529,Emanuel Terry,PHO,"$85,578",NaN,NaN,NaN,NaN,NaN,Minimum Salary,85578.0,1,85578.0
581,530,Tyrell Terry,MEM,"$1,517,981","$1,782,621",NaN,NaN,NaN,NaN,NaN,85578.0,2,42789.0
582,531,Admiral Schofield,ORL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169706.0,0,inf
583,532,Malcolm Hill,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53176.0,0,inf


In [ ]:
# 1. Read the file, specifying comma as the separator
stats25 = pd.read_csv(
    'text25.txt',
    sep=',',          # split on commas
    header=None,      # there is no header row in the file
    encoding='utf-8'  # ensure non-ASCII names load correctly
)
# 2. (Optional) Assign column names
stats25.columns = [
    'Rank','Player','Age','Team','Position','Games','Games Started','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS', 'Player-additional'

]

# 1. Read the file, specifying comma as the separator
stats24 = pd.read_csv(
    'text24.txt',
    sep=',',          # split on commas
    header=None,      # there is no header row in the file
    encoding='utf-8'  # ensure non-ASCII names load correctly
)
# 2. (Optional) Assign column names
stats24.columns = [
    'Rank','Player','Age','Team','Position','Games','Games Started','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS', 'Player-additional'

]


# 1. Read the file, specifying comma as the separator
stats23 = pd.read_csv(
    'text23.txt',
    sep=',',          # split on commas
    header=None,      # there is no header row in the file
    encoding='utf-8'  # ensure non-ASCII names load correctly
)
# 2. (Optional) Assign column names
stats23.columns = [
    'Rank','Player','Age','Team','Position','Games','Games Started','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS', 'Player-additional'

]


# 1. Read the file, specifying comma as the separator
stats22 = pd.read_csv(
    'text22.txt',
    sep=',',          # split on commas
    header=None,      # there is no header row in the file
    encoding='utf-8'  # ensure non-ASCII names load correctly
)
# 2. (Optional) Assign column names
stats22.columns = [
    'Rank','Player','Age','Team','Position','Games','Games Started','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS', 'Player-additional'

]

# 1. Read the file, specifying comma as the separator
stats21 = pd.read_csv(
    'text21.txt',
    sep=',',          # split on commas
    header=None,      # there is no header row in the file
    encoding='utf-8'  # ensure non-ASCII names load correctly
)
# 2. (Optional) Assign column names
stats21.columns = [
    'Rank','Player','Age','Team','Position','Games','Games Started','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS', 'Player-additional'

]



In [ ]:
# (1) make sure the key columns really match:
contract25['Player'] = contract25['Player'].str.strip()
stats25   ['Player'] = stats25   ['Player'].str.strip()

# (2) do an inner‐join on Player
merged25 = pd.merge(
    contract25,
    stats25,
    on='Player',
    how='inner',      # only keeps players present in *both* tables
    suffixes=('_contract', '_stats')
)
display(merged25)

,Rk,Player,Tm,2024-25,2025-26,2026-27,2027-28,2028-29,2029-30,Guaranteed,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional
0,1,Stephen Curry,GSW,"$55,761,216","$59,606,817","$62,587,158",NaN,NaN,NaN,177955191,...,0.6,3.9,4.4,6.0,1.1,0.4,2.9,1.4,24.5,curryst01
1,2,Joel Embiid,PHI,"$51,415,938","$55,224,526","$57,985,752","$62,624,612","$67,263,472",NaN,227250828,...,1.9,6.3,8.2,4.5,0.7,0.9,3.3,2.2,23.8,embiijo01
2,3,Nikola Jokić,DEN,"$51,415,938","$55,224,526","$59,033,114","$62,841,702",NaN,NaN,165673578,...,2.9,9.9,12.7,10.2,1.8,0.6,3.3,2.3,29.6,jokicni01
3,4,Kevin Durant,PHO,"$51,179,021","$54,708,609",NaN,NaN,NaN,NaN,105887630,...,0.4,5.7,6.0,4.2,0.8,1.2,3.1,1.7,26.6,duranke01
4,5,Bradley Beal,PHO,"$50,203,930","$53,666,270","$57,128,610",NaN,NaN,NaN,103870200,...,0.6,2.7,3.3,3.7,1.1,0.5,1.9,2.6,17.0,bealbr01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,489,David Roddy,PHI,"$2,967,212",NaN,NaN,NaN,NaN,NaN,119972,...,0.4,2.2,2.5,1.1,0.4,0.3,0.7,0.6,4.6,roddyda01
465,490,Chuma Okeke,PHI,"$119,972",NaN,NaN,NaN,NaN,NaN,119972,...,2.3,2.9,5.2,1.7,0.6,0.4,0.3,0.6,5.9,okekech01
466,491,Jahlil Okafor,IND,"$119,972",NaN,NaN,NaN,NaN,NaN,119972,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,okafoja01
467,494,Branden Carlson,OKC,"$496,519",NaN,NaN,NaN,NaN,NaN,496519,...,0.5,1.2,1.7,0.4,0.2,0.7,0.2,0.7,3.8,carlsbr01
